In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark-3.3.1-bin-hadoop3'

In [3]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
import datetime
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.master("local").\
                appName("Home_work 3_3").\
                config("spark.driver.bindAddres","localhost").\
                config("spark.ui.port","4040").\
                getOrCreate()

In [161]:
schema_web=T.StructType([
            T.StructField("id",T.StringType(),True),
            T.StructField("timestamp",T.LongType(),True),
            T.StructField("type",T.StringType(),True),
            T.StructField("page_id",T.IntegerType(),True),
            T.StructField("yag",T.StringType(),True),
            T.StructField("sign",T.BooleanType(),True)])

schema_lk=T.StructType([
            T.StructField("id",T.StringType(),True),
            T.StructField("user_id",T.IntegerType(),True),
            T.StructField("fio",T.StringType(),True),
            T.StructField("dob",T.DateType(),True),
            T.StructField("date lk",T.DateType(),True)])

In [162]:
data_web = [(1,1667627426,"visit",247,"Sport",False),
            (1,1668627475,"click",243,"Sport",False),
            (1,1667827488,"click",315,"Medicine",False),
            (1,1664627515,"move",181,"Politics",False),
            (1,1657627560,"scroll",127,"Politics",False),
            (2,1671627670,"click",315,"Medicine",True),
            (2,1673627698,"click",217,"Sport",True),
            (2,1665627723,"click",243,"Sport",True),
            (2,1667627765,"scroll",211,"Sport",True),
            (3,1667627810,"visit",152,"Politics",True),
            (3,1668827905,"move",249,"Sport",True),
            (3,1666228160,"click",274,"Sport",True),
            (4,1667628516,"visit",315,"Medicine",False),
            (4,1669128628,"click",243,"Sport",False),
            (4,1665428790,"click",232,"Sport",False),
            (5,1664828852,"visit",174,"Politics",True),
            (5,1667628917,"scroll",256,"Sport",True),
            (6,1663428917,"visit",315,"Medicine",True),
            (6,1665728917,"click",243,"Sport",True),
            (6,1662128917,"click",687,"Sport",True),
            (6,1666428917,"scroll",256,"Sport",True),
            (6,1665728917,"click",315,"Medicine",True),
            (6,1668928917,"click",122,"Politics",True),
            (6,1669328917,"scroll",357,"Sport",True),
            (6,1668528917,"click",256,"Sport",True),
            (7,1667428917,"visit",844,"Sport",False),
            (7,1669028917,"move",315,"Medicine",False),
            (7,1663928917,"click",122,"Politics",False),
            (7,1664428917,"click",256,"Sport",False)]

In [165]:
data_lk=[(101,1,"Скворцов Юрий Сергеевич",datetime.datetime(1964,9,23),datetime.datetime(2018,4,22)),
         (102,2,"Любимов Дмитрий Кузьмич",datetime.datetime(1964,9,23),datetime.datetime(2018,4,22)),
         (103,3,"Петровва Ольга Александровна",datetime.datetime(1964,9,23),datetime.datetime(2018,4,22)),
         (104,4,"Поляков Егор Владимирович",datetime.datetime(1964,9,23),datetime.datetime(2018,4,22)),
         (105,5,"Маклакова Юлия Сергеевна",datetime.datetime(1964,9,23),datetime.datetime(2018,4,22)),
         (106,6,"Дроздов Антон Елизарович",datetime.datetime(1964,9,23),datetime.datetime(2018,4,22)),
         (107,7,"Перминова Оксана Владиленовна",datetime.datetime(1964,9,23),datetime.datetime(2018,4,22))]

In [166]:
df_web=spark.createDataFrame(data=data_web,schema=schema_web)
df_lk=spark.createDataFrame(data=data_lk,schema=schema_lk)

In [141]:
print("Топ 5 пользователей по количеству событий")

df_web.groupby("id")\
        .count()\
        .orderBy("count",ascending=False)\
        .limit(5)\
        .show()

Топ 5 пользователей по количеству событий
+---+-----+
| id|count|
+---+-----+
|  6|    8|
|  1|    5|
|  7|    4|
|  2|    4|
|  3|    3|
+---+-----+



In [75]:
df_cnt_all=df_web.agg(F.countDistinct("id").alias("count_all"))
df_cnt_tru=df_web.select("sign","id")\
      .where(df_web.sign==True)\
      .agg(F.countDistinct("id").alias("count_tru"))
cnt_tru=df_cnt_tru.head()["count_tru"]
cnt_all=df_cnt_all.head()["count_all"]
percent=cnt_tru/cnt_all*100
print("Пользователей с личным кабинетом:", int(percent),"%")

Пользователей с личным кабинетом: 57 %


In [138]:
print("Топ 5 страниц по количеству кликов")

df_web.where(df_web.type=="click")\
        .orderBy("page_id",ascending=False)\
        .groupby("page_id")\
        .count()\
        .orderBy("count",ascending=False)\
        .limit(5)\
        .show()

Топ 5 страниц по количеству кликов
+-------+-----+
|page_id|count|
+-------+-----+
|    243|    4|
|    315|    3|
|    122|    2|
|    256|    2|
|    232|    1|
+-------+-----+



In [172]:
print("События в диапазонах суток")
df_web=df_web.select(*[i for i in df_web.columns if i !="timestamp"],
                       F.from_unixtime("timestamp").alias("event_time"))
df_web.withColumn("day_period",F.floor(F.hour("event_time")/F.lit(4)+1)).show(30)

События в диапазонах суток
+---+------+-------+--------+-----+-------------------+----------+
| id|  type|page_id|     yag| sign|         event_time|day_period|
+---+------+-------+--------+-----+-------------------+----------+
|  1| visit|    247|   Sport|false|2022-11-05 08:50:26|         3|
|  1| click|    243|   Sport|false|2022-11-16 22:37:55|         6|
|  1| click|    315|Medicine|false|2022-11-07 16:24:48|         5|
|  1|  move|    181|Politics|false|2022-10-01 15:31:55|         4|
|  1|scroll|    127|Politics|false|2022-07-12 15:06:00|         4|
|  2| click|    315|Medicine| true|2022-12-21 16:01:10|         5|
|  2| click|    217|   Sport| true|2023-01-13 19:34:58|         5|
|  2| click|    243|   Sport| true|2022-10-13 05:22:03|         2|
|  2|scroll|    211|   Sport| true|2022-11-05 08:56:05|         3|
|  3| visit|    152|Politics| true|2022-11-05 08:56:50|         3|
|  3|  move|    249|   Sport| true|2022-11-19 06:18:25|         2|
|  3| click|    274|   Sport| true|

In [156]:
print("Период суток с максимальным количеством событий")

hit_ev_period=df_web.withColumn("day_period",F.floor(F.hour("event_time")/F.lit(4)+1))

hit_ev_period.groupby("day_period")\
        .count()\
        .orderBy("count",ascending=False)\
        .limit(1)\
        .show()

Период суток с максимальным количеством событий
+----------+-----+
|day_period|count|
+----------+-----+
|         3|   10|
+----------+-----+



In [24]:
df_web.groupby("id")\
        .agg({
                "type": "count",
                "page_id": "max"    
            })\
        .show(20)

+---+------------+-----------+
| id|max(page_id)|count(type)|
+---+------------+-----------+
|  7|         844|          4|
|  3|         274|          3|
|  5|         256|          2|
|  6|         687|          8|
|  1|         315|          5|
|  4|         315|          3|
|  2|         315|          4|
+---+------------+-----------+



In [171]:
print("Пользователи, заходившие на страницы раздела Sport")
df_all=df_lk.alias("lk").join(df_web.alias("web"),
                            on=[F.col("lk.user_id")==F.col("web.id")],
                            how="left")
df_all.where(df_all.yag=="Sport")\
        .orderBy("fio",ascending=False)\
        .groupby("fio")\
        .count()\
        .orderBy("count",ascending=False)\
        .show()

Пользователи, заходившие на страницы раздела Sport
+--------------------+-----+
|                 fio|count|
+--------------------+-----+
|Дроздов Антон Ели...|    5|
|Любимов Дмитрий К...|    3|
|Поляков Егор Влад...|    2|
|Петровва Ольга Ал...|    2|
|Перминова Оксана ...|    2|
|Скворцов Юрий Сер...|    2|
|Маклакова Юлия Се...|    1|
+--------------------+-----+

